# Notebook pour créer des courbes de lumière
En utilisant les fonctions de calibration et de photométrie disponibles, on va créer simplement une courbe de lumière à partir des images d'un dossier (correspondant à une cible).

In [1]:
import os, time
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import TimeSeries
from scipy.ndimage import median_filter

In [2]:
target_type = "exoplanet"
if target_type == "asteroid":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
elif target_type == "exoplanet":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_EXPLTS)
os.listdir(data_dir)

['2021-10-05_OMICRON_F3p17_OPF_QHY600Ma_Corot-1',
 '2021-10-11_OMICRON_F3p17_OPF_QHY600Ma_Qatar-3',
 '2021-10-14_OMICRON_F3p17_OPF_QHY600Ma_HAT-P-54',
 '2023-11-17_OMICRON_F3p17_OPF_QHY600Ma_TOI3657',
 '2023-11-21_OMICRON_F3p17_OPF_QHY600Ma_TOI3657',
 'DARKS',
 'Readme.md',
 'WASP43_Full',
 'WASP43_incomplete',
 'wd1856+534']

In [3]:
for ddir in os.listdir(data_dir):
    subdata_dir = os.path.join(data_dir, ddir)
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(subdata_dir, len(list_fits))

/mnt/d/Archives_C2PU/Photometry/ExoPInts/2021-10-05_OMICRON_F3p17_OPF_QHY600Ma_Corot-1 283
/mnt/d/Archives_C2PU/Photometry/ExoPInts/2021-10-11_OMICRON_F3p17_OPF_QHY600Ma_Qatar-3 799
/mnt/d/Archives_C2PU/Photometry/ExoPInts/2021-10-14_OMICRON_F3p17_OPF_QHY600Ma_HAT-P-54 629
/mnt/d/Archives_C2PU/Photometry/ExoPInts/2023-11-17_OMICRON_F3p17_OPF_QHY600Ma_TOI3657 619
/mnt/d/Archives_C2PU/Photometry/ExoPInts/2023-11-21_OMICRON_F3p17_OPF_QHY600Ma_TOI3657 607
/mnt/d/Archives_C2PU/Photometry/ExoPInts/DARKS 15
/mnt/d/Archives_C2PU/Photometry/ExoPInts/WASP43_Full 302
/mnt/d/Archives_C2PU/Photometry/ExoPInts/WASP43_incomplete 97
/mnt/d/Archives_C2PU/Photometry/ExoPInts/wd1856+534 46


In [4]:
achoice = os.listdir(data_dir)[0]
aligned = False
subdata_dir = os.path.join(data_dir, achoice)
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
list_fits = sorted(list_fits)

In [5]:
list_fits

['Corot-1_20211006T021147076_SC__0030s000_000000.fits',
 'Corot-1_20211006T021220057_SC__0030s000_000000.fits',
 'Corot-1_20211006T021253055_SC__0030s000_000000.fits',
 'Corot-1_20211006T021325813_SC__0030s000_000000.fits',
 'Corot-1_20211006T021358805_SC__0030s000_000000.fits',
 'Corot-1_20211006T021431684_SC__0030s000_000000.fits',
 'Corot-1_20211006T021504534_SC__0030s000_000000.fits',
 'Corot-1_20211006T021537498_SC__0030s000_000000.fits',
 'Corot-1_20211006T021610460_SC__0030s000_000000.fits',
 'Corot-1_20211006T021643333_SC__0030s000_000000.fits',
 'Corot-1_20211006T021716299_SC__0030s000_000000.fits',
 'Corot-1_20211006T021749239_SC__0030s000_000000.fits',
 'Corot-1_20211006T021822223_SC__0030s000_000000.fits',
 'Corot-1_20211006T021855145_SC__0030s000_000000.fits',
 'Corot-1_20211006T021928178_SC__0030s000_000000.fits',
 'Corot-1_20211006T022001086_SC__0030s000_000000.fits',
 'Corot-1_20211006T022034120_SC__0030s000_000000.fits',
 'Corot-1_20211006T022107135_SC__0030s000_000000

In [6]:
rerun = True
test_mode = False  # Only runs on a few images, prints the tables at each step and does not write files.
dict_of_dict = {}

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter("ignore", category=AstropyWarning)

if rerun or test_mode:
    from tqdm import tqdm

    if test_mode:
        list_fits = list_fits[:10]
    for loc, scimage in enumerate(tqdm(list_fits)):
        fits_sci_image = os.path.join(subdata_dir, scimage)
        fits_sci_image = os.path.abspath(fits_sci_image)
        if os.path.isfile(fits_sci_image):
            # Get image directory, failename and extension
            sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
            sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

            bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)

            # Get information from FITS header
            sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=False)
            # print(sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y)

            # Run calibration
            dico_calib = calibration.reduce_sci_image(fits_sci_image, darks_dir, flats_dir, path_to_bias_dir="", use_bias=False, override_date_check=True, max_days=7, speedup=test_mode, overwrite=rerun, verbose=test_mode, write_tmp=test_mode)
            dict_of_dict.update({sc_im_name: dico_calib})

100%|███████████████████████████████████████████████████████████████████████████████| 283/283 [1:14:46<00:00, 15.85s/it]


Calibration terminée !